In [1]:
# === Required Libraries ===
# Run this once if needed:
# pip install pingouin statsmodels scipy pandas

import pandas as pd
from statsmodels.stats.anova import AnovaRM
from scipy.stats import ttest_rel, shapiro
from statsmodels.stats.multitest import multipletests
import pingouin as pg
from itertools import combinations

# === Step 1: Load and combine data from all token levels ===
df_500k = pd.read_csv("FUPowerResults/500000_tokens/run_log.csv")
df_500k["token_count"] = 500000

df_1m = pd.read_csv("FUPowerResults/1000000_tokens/run_log.csv")
df_1m["token_count"] = 1000000

df_2m = pd.read_csv("FUPowerResults/2000000_tokens/run_log.csv")
df_2m["token_count"] = 2000000

df_all = pd.concat([df_500k, df_1m, df_2m], ignore_index=True)
df_all = df_all[["trial_number", "token_count", "parameter_efficiency"]]

# === Step 2: Run Repeated Measures ANOVA ===
anova = AnovaRM(df_all, depvar="parameter_efficiency", subject="trial_number", within=["token_count"]).fit()
print("\n=== Repeated Measures ANOVA ===")
print(anova)

# === Step 3: Prepare data for pairwise testing and assumptions ===
df_wide = df_all.pivot(index="trial_number", columns="token_count", values="parameter_efficiency")
pairs = list(combinations(df_wide.columns, 2))

# === Step 4: Pairwise Bonferroni-Corrected Dependent T-Tests ===
print("\n=== Pairwise Comparisons (Bonferroni-Corrected) ===")
p_values = []
results = []

for a, b in pairs:
    t_stat, p = ttest_rel(df_wide[a], df_wide[b])
    p_values.append(p)
    results.append((a, b, t_stat, p))

reject, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='bonferroni')

for i, (a, b, t_stat, raw_p) in enumerate(results):
    print(f"{a} vs {b} → t = {t_stat:.4f}, raw p = {raw_p:.4f}, corrected p = {pvals_corrected[i]:.4f}, reject H₀: {reject[i]}")

# === Step 5: Normality Check (Shapiro-Wilk) ===
print("\n=== Shapiro-Wilk Normality Test ===")
for token in df_wide.columns:
    stat, p = shapiro(df_wide[token])
    print(f"{token}: W = {stat:.4f}, p = {p:.4f} → {'Normal' if p > 0.05 else 'Non-normal'}")

# === Step 6: Mauchly’s Test for Sphericity ===
print("\n=== Mauchly’s Test for Sphericity ===")
df_long = pd.melt(df_wide.reset_index(), id_vars='trial_number', var_name='token_count', value_name='efficiency')
df_long['token_count'] = df_long['token_count'].astype(str)

sphericity_test = pg.sphericity(data=df_long, dv='efficiency', subject='trial_number', within='token_count')
print(sphericity_test)

# === Step 7: Detailed ANOVA with Pingouin (Optional) ===
print("\n=== Repeated Measures ANOVA (Pingouin) ===")
aov_pg = pg.rm_anova(dv='efficiency', within='token_count', subject='trial_number', data=df_long, detailed=True)
print(aov_pg)


=== Repeated Measures ANOVA ===
                   Anova
             F Value   Num DF  Den DF Pr > F
--------------------------------------------
token_count 24268.2288 2.0000 98.0000 0.0000


=== Pairwise Comparisons (Bonferroni-Corrected) ===
500000 vs 1000000 → t = 117.9388, raw p = 0.0000, corrected p = 0.0000, reject H₀: True
500000 vs 2000000 → t = 220.8192, raw p = 0.0000, corrected p = 0.0000, reject H₀: True
1000000 vs 2000000 → t = 97.6873, raw p = 0.0000, corrected p = 0.0000, reject H₀: True

=== Shapiro-Wilk Normality Test ===
500000: W = 0.9755, p = 0.3796 → Normal
1000000: W = 0.9735, p = 0.3200 → Normal
2000000: W = 0.9880, p = 0.8872 → Normal

=== Mauchly’s Test for Sphericity ===
SpherResults(spher=True, W=inf, chi2=-inf, dof=2, pval=1.0)

=== Repeated Measures ANOVA (Pingouin) ===
        Source            SS  DF            MS             F          p-unc  \
0  token_count  5.316590e-20   2  2.658295e-20  24268.228843  8.124898e-133   
1        Error  1.073473e-22 

/opt/conda/lib/python3.12/site-packages/pingouin/distribution.py:1004: RuntimeWarning: divide by zero encountered in scalar divide
  W = np.prod(eig) / (eig.sum() / d) ** d
/opt/conda/lib/python3.12/site-packages/pingouin/distribution.py:1004: RuntimeWarning: divide by zero encountered in scalar divide
  W = np.prod(eig) / (eig.sum() / d) ** d


In [7]:
# === Required Libraries ===
# Run this once if needed:
# pip install pingouin statsmodels scipy pandas

import pandas as pd
from statsmodels.stats.anova import AnovaRM
from scipy.stats import ttest_rel, shapiro
from statsmodels.stats.multitest import multipletests
import pingouin as pg
from itertools import combinations

# === Step 1: Load and combine data from all token levels ===
df_500k = pd.read_csv("FUPowerResults/500000_tokens/run_log.csv")
df_500k["token_count"] = 500000

df_1m = pd.read_csv("FUPowerResults/1000000_tokens/run_log.csv")
df_1m["token_count"] = 1000000

df_2m = pd.read_csv("FUPowerResults/2000000_tokens/run_log.csv")
df_2m["token_count"] = 2000000

df_all = pd.concat([df_500k, df_1m, df_2m], ignore_index=True)
df_all = df_all[["trial_number", "token_count", "watts_rms"]]

# === Step 2: Run Repeated Measures ANOVA ===
anova = AnovaRM(df_all, depvar="watts_rms", subject="trial_number", within=["token_count"]).fit()
print("\n=== Repeated Measures ANOVA ===")
print(anova)

# === Step 3: Prepare data for pairwise testing and assumptions ===
df_wide = df_all.pivot(index="trial_number", columns="token_count", values="watts_rms")
pairs = list(combinations(df_wide.columns, 2))

# === Step 4: Pairwise Bonferroni-Corrected Dependent T-Tests ===
print("\n=== Pairwise Comparisons (Bonferroni-Corrected) ===")
p_values = []
results = []

for a, b in pairs:
    t_stat, p = ttest_rel(df_wide[a], df_wide[b])
    p_values.append(p)
    results.append((a, b, t_stat, p))

reject, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='bonferroni')

for i, (a, b, t_stat, raw_p) in enumerate(results):
    print(f"{a} vs {b} → t = {t_stat:.4f}, raw p = {raw_p:.4f}, corrected p = {pvals_corrected[i]:.4f}, reject H₀: {reject[i]}")

# === Step 5: Normality Check (Shapiro-Wilk) ===
print("\n=== Shapiro-Wilk Normality Test ===")
for token in df_wide.columns:
    stat, p = shapiro(df_wide[token])
    print(f"{token}: W = {stat:.4f}, p = {p:.4f} → {'Normal' if p > 0.05 else 'Non-normal'}")

# === Step 6: Mauchly’s Test for Sphericity ===
print("\n=== Mauchly’s Test for Sphericity ===")
df_long = pd.melt(df_wide.reset_index(), id_vars='trial_number', var_name='token_count', value_name='watts_rms')
df_long['token_count'] = df_long['token_count'].astype(str)

sphericity_test = pg.sphericity(data=df_long, dv='watts_rms', subject='trial_number', within='token_count')
print(sphericity_test)

# === Step 7: Detailed ANOVA with Pingouin (Optional) ===
print("\n=== Repeated Measures ANOVA (Pingouin) ===")
aov_pg = pg.rm_anova(dv='watts_rms', within='token_count', subject='trial_number', data=df_long, detailed=True)
print(aov_pg)


=== Repeated Measures ANOVA ===
                   Anova
             F Value  Num DF  Den DF Pr > F
-------------------------------------------
token_count 3143.2063 2.0000 98.0000 0.0000


=== Pairwise Comparisons (Bonferroni-Corrected) ===
500000 vs 1000000 → t = -40.6892, raw p = 0.0000, corrected p = 0.0000, reject H₀: True
500000 vs 2000000 → t = -84.1646, raw p = 0.0000, corrected p = 0.0000, reject H₀: True
1000000 vs 2000000 → t = -36.6477, raw p = 0.0000, corrected p = 0.0000, reject H₀: True

=== Shapiro-Wilk Normality Test ===
500000: W = 0.9799, p = 0.5475 → Normal
1000000: W = 0.8175, p = 0.0000 → Non-normal
2000000: W = 0.9487, p = 0.0300 → Non-normal

=== Mauchly’s Test for Sphericity ===
SpherResults(spher=False, W=0.4218734064626523, chi2=41.42639974253711, dof=2, pval=1.01011679792576e-09)

=== Repeated Measures ANOVA (Pingouin) ===
        Source           SS  DF           MS            F         p-unc  \
0  token_count  3360.174036   2  1680.087018  3143.206296  1